# Plot and compute NIRCam imaging exposure map

using .pointing file exported from APT

for MODULE=ALL case

Mario Gennaro  
Dan Coe

Updates desired:
* automate PRD path definition to where it's installed
* select observations to be plotted / ignore non-NIRCam

Updates made:
* depth in rectangular array, rather than MCMC
* exposure colormap updated; 0 shows up as white, not black (to distinguish from 1)

In [ ]:
import pandas as pd
import numpy as np
import os,glob,itertools
from pysiaf.iando import read
from pysiaf.utils import tools, compare
from pysiaf import siaf,JwstAperture

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib import colors

%matplotlib notebook

plt.style.use('http://www.stsci.edu/~dcoe/matplotlibrc.txt')

## User defined inputs

In [ ]:
channel = 'SW'
module = ['A','B']
ref_aperture_name = 'NRCALL_FULL'

pointing_file = '2198_2mosaics.pointing'
observations = ['1']  # to be plotted (e.g., "Obs 1" in APT file)

#ntrials_arcmin_sq = 100000  # this is the number per arcminute squared
nx = ny = 1000  # samples along each axis for exposure map
ntrials = nx * ny

obs_string=observations[0]
for i in range(1,len(observations)):
    obs_string+='_'+observations[i]

file_suffix = channel+'_'+obs_string
print(file_suffix)

## Global varaibles

### Paths and filenames

In [ ]:
### matplotlib stuff
codes_one_sca = [Path.MOVETO,Path.LINETO,Path.LINETO,Path.LINETO,Path.CLOSEPOLY]

### Pick the PRD version and get corresponding SIAF and subarray files

In [ ]:
dirpath = '/path/to/PRD/data/'
PRD = '/PRDOPSSOC-036/'
siaf_file = dirpath+PRD+'/SIAFXML/SIAFXML/NIRCam_SIAF.xml'

## Ingest the apertures in the right format

### Read in the file into a pysiaf.siaf object

In [ ]:
aperture_collection_1 = read.read_jwst_siaf(filename=siaf_file)
siaf_object = siaf.Siaf('NIRCam')
siaf_object.apertures = aperture_collection_1
siaf_object.description = os.path.basename(siaf_file)
siaf_object.observatory = 'JWST'

### Pick the channel

In [ ]:
if channel == 'SW':
    scas = ['1','2','3','4']
elif channel == 'LW':
    scas = ['5']

### Get the apertures

In [ ]:
ap_names_list = ['NRC'+mod+sca+'_FULL' for mod in module for sca in scas]
apertures = [siaf_object.apertures[apname] for apname in ap_names_list]

ref_aperture = siaf_object.apertures[ref_aperture_name]
print(ap_names_list)

### Get the aperture corners in Ideal coordinates w.r.t. the refrence aperture

The resulting list is a list of list of tuples, one list per sca, each sca list made up of 4 tuples for the 4 corners.

In [ ]:
corner_list = []
for ap in apertures:
    clist = []
    cx_tel,cy_tel = ap.corners('tel')
    
    for tel_x,tel_y in zip(cx_tel,cy_tel):
        idl_x,idl_y = ref_aperture.tel_to_idl(tel_x,tel_y)
        clist.append((idl_x,idl_y))
    corner_list.append(clist)

In [ ]:
codes = []

for sca in scas:
    for mod in module:
        codes.extend(codes_one_sca)

In [ ]:
flattened_vertices = []
flattened_vertices_per_sca = []
path_per_sca = []

for corners in corner_list:
    corners_per_sca = []
    for v in corners:
        flattened_vertices.append([v[0],v[1]])
        corners_per_sca.append([v[0],v[1]])
    flattened_vertices.append([corners[0][0],corners[0][1]])
    corners_per_sca.append(([corners[0][0],corners[0][1]]))
    flattened_vertices_per_sca.append(corners_per_sca)
    path_per_sca.append(Path(np.asarray(corners_per_sca), codes_one_sca)) 
    
path=Path(np.asarray(flattened_vertices), codes)


## Plot a single NIRCam pointing

two different ways

In [ ]:
plt.close("all")
f,ax = plt.subplots(1,1,figsize=(10,4))
patch = patches.PathPatch(path, facecolor='orange', lw=2,alpha=0.8)

ax.scatter(0,0,s=50,marker='H',c='r')
ax.add_patch(patch);
ax.axis('equal');

In [ ]:
plt.close("all")
f,ax = plt.subplots(1,1,figsize=(10,4))

for p in path_per_sca:
    patch = patches.PathPatch(p, facecolor='orange', lw=2,alpha=0.8)
    ax.add_patch(patch)

ax.scatter(0,0,s=50,marker='H',c='r');
ax.axis('equal');

## Parse the pointing file to get the IdlX, IdlY offsets

### Slice the pointing info by Observation (and if require by other levels of the multiindex, e.g. exposure - if you just want to deal with one filter)
Remember, the levels are: ('Observation','Visit','Tar','Tile','Exp','Dith')

In [ ]:
def read_pointing(filename, observations):

    filename = pointing_file

    names = 'Tar Tile Exp Dith Aperture Name Target RA Dec BaseX BaseY DithX DithY V2 V3 IdlX IdlY Level Type ExPar DkPar dDist'.split()          

    df = pd.DataFrame(columns=['Observation','Visit']+names)
    df = df.set_index(['Observation','Visit','Tar','Tile','Exp','Dith'])

    obs = None
    vis = None
    with open(filename) as fp:
        for line in fp:
            if line[:13] == '* Observation':
                obs = line[13:].strip()
                #print('Observation', obs)
            if line[0] =='*' and '(Obs' in line:
                obs = line.split('Obs')[-1].strip()[:-1]
                #print('Obs', obs)
            if line[:8] == '** Visit':
                vis = line.strip().split(':')[1]
                #print('Visit', vis)
            #if (obs is not None) and (vis is not None):
            if (obs in observations) and (vis is not None):
                if (line[:3] != 'Tar') & (len(line) > 100):
                    _list = line.split()
                    if _list[-1] == '(base)':
                        _list = _list[:-1]
                    df.loc[(obs,vis,_list[0],_list[1],_list[2],_list[3]),:] = _list[4:]

    return df

In [ ]:
df = read_pointing(pointing_file, observations)
myindex = pd.IndexSlice['1',:,:,:,'1',:]

#myindex = pd.IndexSlice[observation,:,:,:,:,:]

df.loc[myindex,:]

In [ ]:
dith_X = df.loc[myindex,['IdlX']].values
dith_Y = df.loc[myindex,['IdlY']].values

full_dithers_list = []
for dx,dy in zip(dith_X,dith_Y):
    dx_f = np.float_(dx[0])
    dy_f = np.float_(dy[0])
    full_dithers_list.append((dx_f,dy_f))

## Combine apertures and dithers

### For each dither position, create a list of corners, shifted by (d_idlX, d_idlY)

Also use this loop to get the minmax (x,y) for the montecarlo

In [ ]:
path_list = []
path_per_sca_list = []

min_x, max_x, min_y, max_y =0.,0.,0.,0.

for dither in full_dithers_list:
    flattened_vertices = [] 
    path_per_sca = []
    for corners in corner_list:
        flattened_vertices_per_sca = []
        for v in corners:
            x,y = v[0]+dither[0],v[1]+dither[1]
            flattened_vertices.append([x,y])
            flattened_vertices_per_sca.append([x,y])
            if x < min_x:
                min_x = x
            if x > max_x:
                max_x = x
            if y < min_y:
                min_y = y
            if y > max_y:
                max_y = y
            
        flattened_vertices.append([corners[0][0]+dither[0],corners[0][1]+dither[1]]) 
        flattened_vertices_per_sca.append([corners[0][0]+dither[0],corners[0][1]+dither[1]]) 
        
        path_per_sca.append(Path(np.asarray(flattened_vertices_per_sca), codes_one_sca))
    path_list.append(Path(np.asarray(flattened_vertices), codes))
    path_per_sca_list.append(path_per_sca)


In [ ]:
print(min_x, max_x, min_y, max_y)

total_area_arcmin_sqaured = (max_x-min_x)*(max_y-min_y)/3600.

#ntrials = np.int_(ntrials_arcmin_sq*total_area_arcmin_sqaured)
#print(ntrials_arcmin_sq,total_area_arcmin_sqaured,ntrials)

print(total_area_arcmin_sqaured, ntrials)

### Plot becasue it's nice

#### Matplotlib patch codes

In [ ]:
codes = []

for sca in scas:
    for mod in module:
        codes.extend(codes_one_sca)

### Plot the dithers

as semi-transparent patches   
no sampling computation required

In [ ]:
f,ax = plt.subplots(1,1,figsize=(8,6))
#ax.scatter(0,0,s=50,marker='H',c='r');
for p in path_list:
    patch = patches.PathPatch(p, facecolor='k', alpha=4./len(path_list), edgecolor='None')
    ax.add_patch(patch)
ax.axis('equal')

plt.xlabel('X Ideal (arcsec)')
plt.ylabel('Y Ideal (arcsec)')

#f.savefig('Coverage_Map_'+file_suffix+'.pdf')

## Use a grid to compute the coverage

In [ ]:
yyy, xxx = np.mgrid[min_y:max_y:ny*1j, min_x:max_x:nx*1j]
P = np.array([xxx.ravel(), yyy.ravel()]).T

In [ ]:
# calculate # exposures for each point in grid (takes a minute)
exposed = []
for i,p in enumerate(path_list):
    exposed1 = p.contains_points(P)
    exposed1.shape = ny, nx
    exposed.append(exposed1)

In [ ]:
exposed = np.array(exposed)
n_exposures = np.sum(exposed, axis=0)
nexp, counts = np.unique(n_exposures, return_counts=True)

In [ ]:
# You may want to save the exposure map if it took a long time to calculate
from astropy.io import fits
fits.PrimaryHDU(n_exposures).writeto('exposure_map_'+file_suffix+'.fits', overwrite=True)

In [ ]:
print('exposures, percentage, area (sq arcmin)')
for iexp in nexp:
    frac = counts[iexp] / np.sum(counts)
    print(iexp, ' %2d%%' % np.round(100*frac), '%4.1f' % (counts[iexp]/ntrials*total_area_arcmin_sqaured))
    if frac > 0.01:
        max_exp = iexp

In [ ]:
f,ax = plt.subplots(1,1,figsize=(9,6))

cmap = plt.cm.gnuplot
cmap = plt.cm.CMRmap_r
norm = colors.BoundaryNorm(np.arange(-0.5, max_exp+1), cmap.N)

#scatter = ax.scatter(P[BM,0],P[BM,1], c=n_exps[BM], cmap=cmap,norm=norm, s=1, edgecolor='none')
extent = min_x, max_x, min_y, max_y
im = plt.imshow(n_exposures, cmap=cmap, extent=extent, norm=norm)
colorbar = f.colorbar(im, ticks=np.arange(max_exp+1))
colorbar.set_label('Exposures', rotation=90, labelpad=10)

ax.axis('equal')
plt.ylim(1.2*min_y, 1.2*max_y)  # add some whitespace margin around exposure map

plt.xlabel('X Ideal (arcsec)')
plt.ylabel('Y Ideal (arcsec)')

f.savefig('exposure_map_'+file_suffix+'.png')